In [1]:
from loadTDMS import *
import numpy as np
import matplotlib.pyplot as plt
# from tqdm import tqdm
# from fft_test import *
# from fft_test_not2power import *

In [9]:
%matplotlib qt

path = rf'Z:\100 Raw_data\20221027---切削実験\AE\Cutting\AE_2.tdms'
d = load_tdms(path, ch_num=3)
d = d[0]
d = d[:25000]
x = np.concatenate([d,np.zeros(2**15 - 25000)])
print(len(x))
print(2**15)

plt.plot(d)
# plt.plot(x)

32768
32768


In [3]:
def FFT_main(t, x, dt, split_t_r, overlap, window_F):
#   FFT_main(t, d, dt, split_t_r, overlap, window_F)
#       
    
    # データをオーバーラップして分割する。
    split_data = data_split(t, x, split_t_r, overlap)

    # FFTを行う。
    FFT_result_list = []
    for split_data_cont in split_data:
        FFT_result_cont = FFT(split_data_cont, dt, window_F)
        FFT_result_list.append(FFT_result_cont)

    # 平均化
    fq_ave = FFT_result_list[0][0]
    F_abs_amp_ave = np.zeros(len(fq_ave))
    for i in range(len(FFT_result_list)):
        F_abs_amp_ave = F_abs_amp_ave + FFT_result_list[i][1]
    F_abs_amp_ave = F_abs_amp_ave/(i+1)

    return fq_ave, F_abs_amp_ave


def FFT(data_input, dt, window_F):

    N = len(data_input[0])
    
    # 窓の用意
    if window_F == "hanning":
        window = np.hanning(N)          # ハニング窓
    elif window_F == "hamming":
        window = np.hamming(N)          # ハミング窓
    elif window_F == "blackman":
        window = np.blackman(N)         # ブラックマン窓
    else:
        print("Error: input window function name is not sapported. Your input: ", window_F)
        print("Hanning window function is used.")
        hanning = np.hanning(N)          # ハニング窓


    # 窓関数後の信号
    x_windowed = data_input[1]*window

    # FFT計算
    F = np.fft.fft(x_windowed)
    F_abs = np.abs(F)
    F_abs_amp = F_abs / N * 2
    fq = np.linspace(0, 1.0/dt, N)
#     fq = np.linspace(0, N*dt, N)

    # 窓補正
    acf = 1/(sum(window)/N)
    F_abs_amp = acf*F_abs_amp

    # ナイキスト定数まで抽出
    fq_out = fq[:int(N/2)+1]
    F_abs_amp_out = F_abs_amp[:int(N/2)+1]

    return [fq_out, F_abs_amp_out]

def data_split(t, x, split_t_r, overlap):

    split_data = []
    one_frame_N = int(len(t)*split_t_r)  # 1フレームのサンプル数
    overlap_N = int(one_frame_N*overlap)  # オーバーラップするサンプル数
    start_S = 0
    end_S = start_S + one_frame_N

    while True:
        t_cont = t[start_S:end_S]
        x_cont = x[start_S:end_S]
        split_data.append([t_cont, x_cont])

        start_S = start_S + (one_frame_N - overlap_N)
        end_S = start_S + one_frame_N

        if end_S > len(t):
            break

    return np.array(split_data)

In [5]:
dt = 1/10**6
split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
overlap = 0.5  # オーバーラップ率
window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
y_label = "amplitude"
y_unit = "V"
N_d = len(d)
N_x = len(x)
t_d = np.linspace(0, N_d*dt, N_d)  # 周波数軸　linspace(開始,終了,分割数)
t_x = np.linspace(0, N_x*dt, N_x)

fq_d, amp_d = FFT_main(t_d, d, dt, split_t_r, overlap, window_F)
fq_x, amp_x = FFT_main(t_x, x, dt, split_t_r, overlap, window_F)

In [6]:
print(len(amp_d))
print(len(amp_x))

1251
1639


In [7]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.xlabel('freqency[kHz]', fontsize=16)
plt.ylabel(y_label+"[dB]", fontsize=16)
ax.set_xlim(0, 500000)
ax.set_xticklabels([0, 100, 200, 300, 400, 500])
ax.set_ylim(-120, 0)
ax.set_yticklabels([-120,-100, -80, -60, -40, -20,0])
# plt.title(rf'{num}path')

plt.plot(fq_d, 20 * np.log10(amp_d))
plt.plot(fq_x, 20 * np.log10(amp_x))

ax.grid()
p = plt.tick_params(labelsize=16)
plt.tight_layout()

In [8]:
%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111)
plt.xlabel('freqency[kHz]', fontsize=16)
plt.ylabel(y_label+"[dB]", fontsize=16)
ax.set_xlim(0, 500000)
ax.set_xticklabels([0, 100, 200, 300, 400, 500])
ax.set_ylim(-120, 0)
ax.set_yticklabels([-120,-100, -80, -60, -40, -20,0])
# plt.title(rf'{num}path')

plt.plot(fq_d, 20 * np.log10(amp_d), label="25000")
plt.plot(fq_x, 20 * np.log10(amp_x), label="zero padding")

ax.grid()
p = plt.tick_params(labelsize=16)
plt.tight_layout()
plt.legend()